In [180]:
from collections import defaultdict, namedtuple
from itertools import product

TraceCell = namedtuple('TraceCell', ['T', 'text', 'p'])


class PCFG:
    
    def __init__(self, production_rules: list = None, start_symbol: NonTerminal = None, probability_of_rules: dict = None):
        
#         self.non_terminals = non_terminals if non_terminals is not None else []
#         self.terminals = terminals if terminals is not None else []
        
        rules, start_symbol = to_cnf([production_rules, start_symbol])
        print(rules, start_symbol)
        self.lexicon = defaultdict(list)
        self.rules = []
        for rule in rules:
            if isinstance(rule.RHS[0], Terminal):
                self.lexicon[rule.RHS[0].name].append((rule.LHS, rule.p))
            else:
                self.rules.append(rule)

        self.production_rules = rules
        self.start_symbol = start_symbol
        self.probability_of_rules = probability_of_rules if probability_of_rules is not None else {}
        
    def parse(self, sentence):
        
        words = sentence.split()
        
        trace = defaultdict(list)
        n = len(words) + 1
        
        for i in range(1, n):
            word = words[i-1]
            trace[(i-1, i)] = [TraceCell(term, word, p) for (term, p) in self.lexicon[word]]
            
        print(trace)
        for j in range(2, n):
            for i in range(j-1, -1, -1):
                for k in range(i+1, j):
                    for c1, c2 in product(trace[(i, k)], trace[(k, j)]):
                        
                        for c3, p in self.use_rules(c1.T, c2.T):
                            trace[(i, j)].append(TraceCell(c3, f"{c1[1]} {c2[1]}", p * c1.p * c2.p))
                            
        return trace[(0, n-1)], trace
    
    def use_rules(self, c1, c2):
        for rule in self.rules:
            left, right = rule.RHS
            parent = rule.LHS

            if left == c1 and right == c2:
                yield parent, rule.p #f"{sem[app_order[0]]}({sem[app_order[1]]})")
                
                
                
                
                

In [179]:
# rules2 = [Rule(S, [A, S, B]),
#             Rule(A, [a, A, S]), 
#             Rule(A, [a]), 
#             Rule(A, []), 
#             Rule(B, [S, B, S]),
#             Rule(B, [A]),
#             Rule(B, [b, b])]

# rules3 = [Rule(S, [A, a]),
#           Rule(S, [B]),
#           Rule(A, [b]),
#           Rule(A, [B]),
#           Rule(B, [A]),
#           Rule(B, [a])]
# g3 = [rules3, S]

# pcfg2 = PCFG(rules3, S)

# pcfg1 = PCFG(rules2, S)
# pcfg1.parse("a a a b b b b")

pcfg3 = PCFG(rules1, S)
out = pcfg3.parse("put a cup on the table")
# pcfg2.parse("b a a")

[DT <- a (0.1), NN <- cup (0.5), DT <- the (0.7), IN <- on (1), NN <- table (0.5), NR0 <- put (1), S <- NR0 NPP (1.0), NR1 <- NP IN (1), NPP <- NR1 NP (1.0), NP <- DT NN (1.0)] S
defaultdict(<class 'list'>, {(0, 1): [TraceCell(T=NR0, text='put', p=1)], (1, 2): [TraceCell(T=DT, text='a', p=0.1)], (2, 3): [TraceCell(T=NN, text='cup', p=0.5)], (3, 4): [TraceCell(T=IN, text='on', p=1)], (4, 5): [TraceCell(T=DT, text='the', p=0.7)], (5, 6): [TraceCell(T=NN, text='table', p=0.5)]})


([TraceCell(T=S, text='put a cup on the table', p=0.017499999999999998)],
 defaultdict(list,
             {(0, 1): [TraceCell(T=NR0, text='put', p=1)],
              (1, 2): [TraceCell(T=DT, text='a', p=0.1)],
              (2, 3): [TraceCell(T=NN, text='cup', p=0.5)],
              (3, 4): [TraceCell(T=IN, text='on', p=1)],
              (4, 5): [TraceCell(T=DT, text='the', p=0.7)],
              (5, 6): [TraceCell(T=NN, text='table', p=0.5)],
              (1, 3): [TraceCell(T=NP, text='a cup', p=0.05)],
              (0, 2): [],
              (2, 4): [],
              (1, 4): [TraceCell(T=NR1, text='a cup on', p=0.05)],
              (0, 3): [],
              (3, 5): [],
              (2, 5): [],
              (1, 5): [],
              (0, 4): [],
              (4, 6): [TraceCell(T=NP, text='the table', p=0.35)],
              (3, 6): [],
              (2, 6): [],
              (1,
               6): [TraceCell(T=NPP, text='a cup on the table', p=0.017499999999999998)],
            

In [119]:
rules2

[S0 <- S, S <- A S B, A <- a A S, A <- a, A <- , B <- S B S, B <- A, B <- b b]

In [173]:
class NonTerminal():
    
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return self.name
    
    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)
        
class Terminal():
    
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return self.name
    
     
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)
        
        
class Rule():
    
    def __init__(self, LHS, RHS, p=1):
        self.LHS = LHS
        self.RHS = RHS
        self.p = p
        
        
    def __repr__(self):
        return f"{self.LHS} <- {' '.join([str(t) for t in self.RHS])} ({self.p})"
    
    def __eq__(self, other):
        return (self.LHS == other.LHS and all([r1 == r2 for r1, r2 in zip(self.RHS, other.RHS)]) 
                and len(self.RHS) == len(other.RHS) and self.p == other.p
               )
        
put = Terminal('put')
a = Terminal('a')
cup = Terminal('cup')
on = Terminal('on')
the = Terminal('the')
table = Terminal('table')

VB = NonTerminal('VB')
DT = NonTerminal('DT')
IN = NonTerminal('IN')
NN = NonTerminal('NN')
NP = NonTerminal('NP')
S = NonTerminal('S')
NPP = NonTerminal("NPP")
PP = NonTerminal('PP')

rules1 = [Rule(VB, [put], 1.0), 
           Rule(DT, [a], 0.1),
           Rule(NN, [cup], 0.5),
           Rule(DT, [the], 0.7),
           Rule(IN, [on], 1),
           Rule(NN, [table], 0.5),
           Rule(S, [put, NPP], 1.0),
           Rule(NPP, [NP, IN, NP], 1.0),
           Rule(NP, [DT, NN], 1.0)]


b = Terminal('b')
A = NonTerminal('A')
B = NonTerminal('B')

rules2 = [Rule(S, [A, S, B], 1),
            Rule(A, [a, A, S], 0.3), 
            Rule(A, [a], 0.2), 
            Rule(A, [], 0.5), 
            Rule(B, [S, B, S], 0.1),
            Rule(B, [A], 0.6),
            Rule(B, [b, b], 0.3)]


g1 = [rules1, S]
g2 = [rules2, S]


def normalise_rule(grammar, lhs):
    rules, s = grammar
    
    to_normalise = [r for r in rules if r.LHS == lhs]
    p_norm = sum([r.p for r in to_normalise])
    if p_norm == 1:
        return grammar
    else:
        new_rules = [Rule(r.LHS, r.RHS, r.p/p_norm) for r in to_normalise]
        
    rules = [r for r in rules if r not in to_normalise] + new_rules
    return (rules, s)
        

def to_cnf(grammar):
    
    added_rule_nr = 0
    
    rule_variable_name = 'NR'
    
    rules, s = grammar
    start = s
    
    for rule in rules:
        lhs = rule.LHS
        rhs = rule.RHS
        if start in rhs:
            s0 = NonTerminal(start.name+"0")
            rules.insert(0, Rule(s0, [start]))
            start = s0

    change = True

    while change:
        change = False
        for rule in rules:
            lhs = rule.LHS
            rhs = rule.RHS
            if rhs == []:
#                 if lhs == start:
#                     continue
                change = True
                new_rules = []
                for rule2 in rules:
                    if rule != rule2:
                        if lhs in rule2.RHS:
                            p = rule.p * rule2.p
                            p2 = (1-rule.p) * rule2.p
                            new_rules.append(Rule(rule2.LHS, [t for t in rule2.RHS if t != lhs], p))
                            rule2 = Rule(rule2.LHS, rule2.RHS, p2)
                        new_rules.append(rule2)
                        
                rules, s = normalise_rule((new_rules, s), lhs)
                
            if len(rhs) == 1 and lhs == rhs[0]:
                change = True
                new_rules = [r for r in rules if r != rule]
                rules = new_rules
                rules, s = normalise_rule((rules, s), rule.LHS)
                
    change = True
    
    guf = []
    
    
    while change:
        change = False
        to_remove = []
        for rule in rules:
            lhs = rule.LHS
            rhs = rule.RHS
            
            
            if len(rhs) == 1 and isinstance(rhs[0], NonTerminal):
                
                to_remove.append(rule)
            else:
                guf.append(rule)
                
        for rule in to_remove:
            change = True
            lhs = rule.LHS
            rhs = rule.RHS[0]

            
            rhsides = [(r.RHS, r.p) for r in rules if r.LHS == rhs]
            
            for (r,p) in rhsides:
                guf.append(Rule(lhs, r, rule.p * p))
        
        removals = [r.LHS for r in guf if r.RHS[0] == r.LHS and len(r.RHS) == 1]
        guf = [r for r in guf if not(r.RHS[0] == r.LHS and len(r.RHS) == 1)]
        
        for l in removals:
            guf, s = normalise_rule((guf, s), l)
        
        rules = guf
        guf = []
    
    rules, start = remove_non_reachable_state([rules, start])
        
    new_terminal_rules = {}
    
    new_rules = []
    for rule in rules:
        lhs = rule.LHS
        rhs = rule.RHS
        
        if len(rhs) > 1 and any([isinstance(t, Terminal) for t in rhs]):
            new_rhs = []
            for t in rhs:
                if isinstance(t, Terminal):
                    if t not in new_terminal_rules:
                        l = NonTerminal(rule_variable_name + str(added_rule_nr))
                        added_rule_nr += 1
                        r = Rule(l, [t], 1)
                        new_rules.append(r)
                        new_terminal_rules[t] = l
                    
                    new_rhs.append(new_terminal_rules[t])
                else:
                    new_rhs.append(t)
            new_rules.append(Rule(lhs, new_rhs, rule.p))
        else:
            new_rules.append(rule)
    
    rules = new_rules
    new_rules = []             
    change = True
    while change:
        change = False
        for rule in rules:
            if len(rule.RHS) > 2:
                change = True
                
                l = NonTerminal(rule_variable_name + str(added_rule_nr))
                added_rule_nr += 1
                
                lhs = rule.LHS
                rhs1 = rule.RHS[:2]
                rhs2 = [l] + rule.RHS[2:]
                
                new_rules.append(Rule(l, rhs1, 1))
                new_rules.append(Rule(lhs, rhs2, rule.p))
                
            else:
                new_rules.append(rule)
    
        rules = new_rules
        new_rules = []
            
    return [rules, start]



print("grammar2 start")
display_grammar(g2)
print("grammar2 end")
display_grammar(to_cnf(g2))



rules3 = [Rule(S, [A, a], 0.6),
          Rule(S, [B], 0.4),
          Rule(A, [b], 0.7),
          Rule(A, [B], 0.3),
          Rule(B, [A], 0.2),
          Rule(B, [a], 0.8)]
g3 = [rules3, S]

print("grammar3 start")
display_grammar(g3)
g = to_cnf(g3)
print("grammar3 end")
display_grammar(g)

print("grammar1 start")
display_grammar(g1)
print("grammar1 end")
display_grammar(to_cnf(g1))


rules4 = [Rule(S, [B, A], 1),
          Rule(A, [a], 0.7),
          Rule(A, [], 0.3),
          Rule(B, [b], 1.0)]
g4 = [rules4, S]
print("grammar4 start")
display_grammar(g4)
print("grammar4 end")
display_grammar(to_cnf(g4))

rules5 = [Rule(S, [a, a], 0.7),
          Rule(S, [b], 0.3),]
g5 = [rules5, S]
print("grammar5 start")
display_grammar(g5)
print("grammar5 end")
display_grammar(to_cnf(g5))

rules5 = [Rule(S, [A, B, B], 0.7),
          Rule(S, [B, B], 0.3),
         Rule(B, [b], 1),
         Rule(A, [a], 1)]
g5 = [rules5, S]
print("grammar5 start")
display_grammar(g5)
print("grammar5 end")
display_grammar(to_cnf(g5))

grammar2 start
S <- A S B (1)
A <- a A S (0.3) | a (0.2) |  (0.5)
B <- S B S (0.1) | A (0.6) | b b (0.3)
grammar2 end
NR0 <- a (1)
A <- NR0 S (0.3) | NR2 S (0.3) | a (0.4)
NR2 <- NR0 A (1)
B <- S S (0.04285714285714286) | NR3 S (0.09999999999999999) | NR1 NR1 (0.4285714285714286) | NR0 S (0.1285714285714286) | NR6 S (0.1285714285714286) | a (0.17142857142857146)
NR3 <- S B (1)
NR1 <- b (1)
S <- S B (0.4117647058823529) | A S (0.17647058823529413) | NR4 B (0.4117647058823529)
NR4 <- A S (1)
S0 <- S B (0.4117647058823529) | A S (0.17647058823529413) | NR5 B (0.4117647058823529)
NR5 <- A S (1)
NR6 <- NR0 A (1)
grammar3 start
S <- A a (0.6) | B (0.4)
A <- b (0.7) | B (0.3)
B <- A (0.2) | a (0.8)
grammar3 end
NR0 <- a (1)
S <- A NR0 (0.6) | a (0.32000000000000006) | b (0.05957446808510639) | a (0.02042553191489362)
A <- b (0.7446808510638298) | a (0.2553191489361702)
grammar1 start
VB <- put (1.0)
DT <- a (0.1) | the (0.7)
NN <- cup (0.5) | table (0.5)
IN <- on (1)
S <- put NPP (1.0)
NPP <-

In [161]:
def display_grammar(g):
    rules, s = g
    
    grammar = defaultdict(list)
    
    for rule in rules:
        grammar[rule.LHS].append((rule.RHS, rule.p))
        
    for key, value in grammar.items():
        print(f"{key} <- {' | '.join([' '.join([str(a) for a in t]) + f' ({p})' for t, p in value])}")

display_grammar(g)

NR0 <- a (1)
S <- A NR0 (1) | a (1) | b (1) | a (1)
A <- b (0.7) | a (1)


In [75]:
def reachable_states(grammar):
    rules, s = grammar
    
    reached_states = []
    
    search(rules, s, reached_states)
    
    return reached_states
    
def search(rules, s, reached_states):
    
    reached_states.append(s)
    
    relevant_rules = [r for r in rules if r.LHS == s]
    
    for r in relevant_rules:
        nodes = next_nodes(r, reached_states)
        
        for node in nodes:
            search(rules, node, reached_states)
        


def next_nodes(rule, reached_states):
    return [t for t in rule.RHS if isinstance(t, NonTerminal) and t not in reached_states]
        
def remove_non_reachable_state(g):
    rs = reachable_states(g)
    
    rules, s = g
    rules = [r for r in rules if r.LHS in rs]
    
    return (rules, s)

remove_non_reachable_state(g)  

([S<-Aa, A<-b, S<-a, A<-a, S<-b, S<-a], S)

In [ ]:
[('put', 'VB'),
 ('a', 'DT'),
 ('cup', 'NN'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('table', 'NN')]

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
sents = nlp(u'a mug should be on it')

sents.to_json()

# def get_words_and_tags(sentence):
#     nlp = spacy.load("en_core_web_sm")
#     sents = nlp(sentence)
    
#     for token in sents['tokens']:
#         word = sents[token['start']:token['end']]
#         tag = token['tag']
        
#         yield (word, tag)

{'text': 'a mug should be on it',
 'ents': [],
 'sents': [{'start': 0, 'end': 21}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 1,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 1},
  {'id': 1,
   'start': 2,
   'end': 5,
   'pos': 'PROPN',
   'tag': 'NNP',
   'dep': 'nsubj',
   'head': 3},
  {'id': 2,
   'start': 6,
   'end': 12,
   'pos': 'VERB',
   'tag': 'MD',
   'dep': 'aux',
   'head': 3},
  {'id': 3,
   'start': 13,
   'end': 15,
   'pos': 'AUX',
   'tag': 'VB',
   'dep': 'ROOT',
   'head': 3},
  {'id': 4,
   'start': 16,
   'end': 18,
   'pos': 'ADP',
   'tag': 'IN',
   'dep': 'prep',
   'head': 3},
  {'id': 5,
   'start': 19,
   'end': 21,
   'pos': 'PRON',
   'tag': 'PRP',
   'dep': 'pobj',
   'head': 4}]}

In [8]:
sents = nlp(u'a green mug should be on the tray')

sents.to_json()

{'text': 'a green mug should be on the tray',
 'ents': [],
 'sents': [{'start': 0, 'end': 33}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 1,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 2},
  {'id': 1,
   'start': 2,
   'end': 7,
   'pos': 'ADJ',
   'tag': 'JJ',
   'dep': 'amod',
   'head': 2},
  {'id': 2,
   'start': 8,
   'end': 11,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'nsubj',
   'head': 4},
  {'id': 3,
   'start': 12,
   'end': 18,
   'pos': 'VERB',
   'tag': 'MD',
   'dep': 'aux',
   'head': 4},
  {'id': 4,
   'start': 19,
   'end': 21,
   'pos': 'AUX',
   'tag': 'VB',
   'dep': 'ROOT',
   'head': 4},
  {'id': 5,
   'start': 22,
   'end': 24,
   'pos': 'ADP',
   'tag': 'IN',
   'dep': 'prep',
   'head': 4},
  {'id': 6,
   'start': 25,
   'end': 28,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 7},
  {'id': 7,
   'start': 29,
   'end': 33,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'pobj',
   'head': 5}]}

In [20]:
def get_words_and_tags(sentence):
    nlp = spacy.load("en_core_web_sm")
    sents = nlp(sentence).to_json()
    
    # print(sents['tokens'])
    
    
    
    for token in sents['tokens']:
        word = sentence[token['start']:token['end']]
        tag = token['tag']
        dep = token['dep']
        pos = token['pos']
        head = token['head']
        yield {'word':word, 'tag':tag, 'dep':dep, 'pos':pos, 'head':head}

In [14]:
sentences = ["An Acura Integra is parked in the lot",
             "There is an Acura Integra parked in the lot.",
             "John parked an Acura Integra in the lot.", 
             "John gave his Acura Integra a bath",
             "Inside his Acura Integra, John showed Susan his new CD player"]

In [22]:
for sentence in sentences:
    for line in get_words_and_tags(sentence):
        print(line)
    print()

{'word': 'An', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 2}
{'word': 'Acura', 'tag': 'NNP', 'dep': 'compound', 'pos': 'PROPN', 'head': 2}
{'word': 'Integra', 'tag': 'NNP', 'dep': 'nsubjpass', 'pos': 'PROPN', 'head': 4}
{'word': 'is', 'tag': 'VBZ', 'dep': 'auxpass', 'pos': 'AUX', 'head': 4}
{'word': 'parked', 'tag': 'VBN', 'dep': 'ROOT', 'pos': 'VERB', 'head': 4}
{'word': 'in', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP', 'head': 4}
{'word': 'the', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 7}
{'word': 'lot', 'tag': 'NN', 'dep': 'pobj', 'pos': 'NOUN', 'head': 5}

{'word': 'There', 'tag': 'EX', 'dep': 'expl', 'pos': 'PRON', 'head': 1}
{'word': 'is', 'tag': 'VBZ', 'dep': 'ROOT', 'pos': 'AUX', 'head': 1}
{'word': 'an', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 4}
{'word': 'Acura', 'tag': 'NNP', 'dep': 'compound', 'pos': 'PROPN', 'head': 4}
{'word': 'Integra', 'tag': 'NNPS', 'dep': 'attr', 'pos': 'PROPN', 'head': 1}
{'word': 'parked', 'tag': 'VBN', 'dep': 'acl', 'pos': 'VER

In [23]:
def get_potential_referents(sentence):
    for word in sentence:
        if 'obj' in word['dep'] or 'subj' in word['dep']:
            yield word

In [13]:
list(get_words_and_tags('a mug should be on it'))

[{'id': 0, 'start': 0, 'end': 1, 'pos': 'DET', 'tag': 'DT', 'dep': 'det', 'head': 1}, {'id': 1, 'start': 2, 'end': 5, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'nsubj', 'head': 3}, {'id': 2, 'start': 6, 'end': 12, 'pos': 'VERB', 'tag': 'MD', 'dep': 'aux', 'head': 3}, {'id': 3, 'start': 13, 'end': 15, 'pos': 'AUX', 'tag': 'VB', 'dep': 'ROOT', 'head': 3}, {'id': 4, 'start': 16, 'end': 18, 'pos': 'ADP', 'tag': 'IN', 'dep': 'prep', 'head': 3}, {'id': 5, 'start': 19, 'end': 21, 'pos': 'PRON', 'tag': 'PRP', 'dep': 'pobj', 'head': 4}]


[{'word': 'a', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 1},
 {'word': 'mug', 'tag': 'NNP', 'dep': 'nsubj', 'pos': 'PROPN', 'head': 3},
 {'word': 'should', 'tag': 'MD', 'dep': 'aux', 'pos': 'VERB', 'head': 3},
 {'word': 'be', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'AUX', 'head': 3},
 {'word': 'on', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP', 'head': 3},
 {'word': 'it', 'tag': 'PRP', 'dep': 'pobj', 'pos': 'PRON', 'head': 4}]

In [11]:
list(get_words_and_tags('put a cup on the table'))

[{'id': 0, 'start': 0, 'end': 3, 'pos': 'VERB', 'tag': 'VB', 'dep': 'ROOT', 'head': 0}, {'id': 1, 'start': 4, 'end': 5, 'pos': 'DET', 'tag': 'DT', 'dep': 'det', 'head': 2}, {'id': 2, 'start': 6, 'end': 9, 'pos': 'NOUN', 'tag': 'NN', 'dep': 'dobj', 'head': 0}, {'id': 3, 'start': 10, 'end': 12, 'pos': 'ADP', 'tag': 'IN', 'dep': 'prep', 'head': 0}, {'id': 4, 'start': 13, 'end': 16, 'pos': 'DET', 'tag': 'DT', 'dep': 'det', 'head': 5}, {'id': 5, 'start': 17, 'end': 22, 'pos': 'NOUN', 'tag': 'NN', 'dep': 'pobj', 'head': 3}]


[{'word': 'put', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB'},
 {'word': 'a', 'tag': 'DT', 'dep': 'det', 'pos': 'DET'},
 {'word': 'cup', 'tag': 'NN', 'dep': 'dobj', 'pos': 'NOUN'},
 {'word': 'on', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP'},
 {'word': 'the', 'tag': 'DT', 'dep': 'det', 'pos': 'DET'},
 {'word': 'table', 'tag': 'NN', 'dep': 'pobj', 'pos': 'NOUN'}]

In [4]:

grammar = PCFG()

In [ ]:
 def parse(self, sentence):
        words = sentence.split()
        
        trace = defaultdict(list)
        n = len(words) + 1
        
        for i in range(1, n):
            word = words[i-1]
            trace[(i-1, i)] = [[(syntax, semantics), word] for syntax, semantics, in self.lexicon[word]]
        
        for j in range(2, n):
            for i in range(j-1, -1, -1):
                for k in range(i+1, j):
                    for c1, c2 in product(trace[(i, k)], trace[(k, j)]):
                        
                        for c3 in self.use_rules(c1[0], c2[0]):
                            trace[(i, j)].append([c3, f"{c1[1]} {c2[1]}"])
                            
        return trace[(0, n-1)]
    
    